In [13]:
import xml.etree.ElementTree as et
import glob
import os
import string
from os import walk
path = "c:/un_dataset/ENGLISH/UNv1.0-TEI/en/2002/opa/ac_33"
paths = [os.path.join(root, name)
        for root, dirs, files in os.walk(path)
             for name in files
             if name.endswith((".xml"))]
#C:\\ENGLISH\UNv1.0-TEI\en

In [14]:
for i in paths:
    print(i)

c:/un_dataset/ENGLISH/UNv1.0-TEI/en/2002/opa/ac_33\1.xml
c:/un_dataset/ENGLISH/UNv1.0-TEI/en/2002/opa/ac_33\2.xml
c:/un_dataset/ENGLISH/UNv1.0-TEI/en/2002/opa/ac_33\24.xml
c:/un_dataset/ENGLISH/UNv1.0-TEI/en/2002/opa/ac_33\26.xml
c:/un_dataset/ENGLISH/UNv1.0-TEI/en/2002/opa/ac_33\3.xml
c:/un_dataset/ENGLISH/UNv1.0-TEI/en/2002/opa/ac_33\4.xml
c:/un_dataset/ENGLISH/UNv1.0-TEI/en/2002/opa/ac_33\5.xml
c:/un_dataset/ENGLISH/UNv1.0-TEI/en/2002/opa/ac_33\6.xml
c:/un_dataset/ENGLISH/UNv1.0-TEI/en/2002/opa/ac_33\9.xml


In [15]:
paths

['c:/un_dataset/ENGLISH/UNv1.0-TEI/en/2002/opa/ac_33\\1.xml',
 'c:/un_dataset/ENGLISH/UNv1.0-TEI/en/2002/opa/ac_33\\2.xml',
 'c:/un_dataset/ENGLISH/UNv1.0-TEI/en/2002/opa/ac_33\\24.xml',
 'c:/un_dataset/ENGLISH/UNv1.0-TEI/en/2002/opa/ac_33\\26.xml',
 'c:/un_dataset/ENGLISH/UNv1.0-TEI/en/2002/opa/ac_33\\3.xml',
 'c:/un_dataset/ENGLISH/UNv1.0-TEI/en/2002/opa/ac_33\\4.xml',
 'c:/un_dataset/ENGLISH/UNv1.0-TEI/en/2002/opa/ac_33\\5.xml',
 'c:/un_dataset/ENGLISH/UNv1.0-TEI/en/2002/opa/ac_33\\6.xml',
 'c:/un_dataset/ENGLISH/UNv1.0-TEI/en/2002/opa/ac_33\\9.xml']

In [16]:
from lxml import etree
parser = etree.XMLParser(recover=True)


outcry = {}
body = ['' for i in range(len(paths))]
outcry['symbol'] = []
outcry['jobno']= []
outcry['dat'] = []
outcry['language'] = []
outcry['keywords'] = [[] for i in range(len(paths))]
outcry['paths'] = []
outcry['text'] = [[] for i in range(len(paths))]
for i in range(len(paths)):       

        tree = tree = et.parse(paths[i],parser=parser)
        root = tree.getroot()
        
        outcry['paths'].append(paths[i])
        
        for idno in root.findall(".//idno") or []:
            if idno.text is not None:
                outcry[idno.get('type')].append(idno.text) 
                
  
        for date in tree.findall('.//date') or []:
            if date.text is not None:
                   outcry['dat'].append(date.text) 
                    
        for lg in tree.findall('.//language')or []:
                if lg.text is not None:
                     outcry['language'].append(lg.text)
                        
        for keyword in root.find(".//keywords") or []:
               if keyword.text is not None:
                    outcry['keywords'][i].append(keyword.text)
                                
                        
        for paragraph in tree.findall(".//body/p"):
            for sentence in paragraph.findall("s"):
                if sentence.text is not None:
                    body[i] += sentence.text
                    if sentence.text[-1:] not in string.punctuation:
                        body[i] += '. '
                    elif sentence == paragraph.findall("s")[-1]:
                        #if it's the last sentence in a paragraph
                        if sentence.text[-1:] in string.punctuation:
                            #replace last element for a period
                            body[i] = body[i][:-1]
                        body[i] += '. '
                    else:
                        body[i] += ' '
                        
                    outcry['text'][i] = body[i]

C:\Users\AlexVaqueroRamirez\Anaconda3\lib\site-packages\ipykernel_launcher.py:34: FutureWarning: The behavior of this method will change in future versions. Use specific 'len(elem)' or 'elem is not None' test instead.


In [17]:
outcry

{'symbol': ['OPA/AC.33/1',
  'OPA/AC.33/2',
  'OPA/AC.33/24',
  'OPA/AC.33/26',
  'OPA/AC.33/3',
  'OPA/AC.33/4',
  'OPA/AC.33/5',
  'OPA/AC.33/6',
  'OPA/AC.33/9'],
 'jobno': ['G0220737',
  'G0224063',
  'G0223491',
  'G0223531',
  'G0222071',
  'G0222075',
  'G0222263',
  'G0222079',
  'G0222083'],
 'dat': ['20020305',
  '20020917',
  '20020802',
  '20020808',
  '20020524',
  '20020531',
  '20020617',
  '20020528',
  '20020531'],
 'language': ['English',
  'English',
  'English',
  'English',
  'English',
  'English',
  'English',
  'English',
  'English'],
 'keywords': [['YOUTH',
   'ENTREPRENEURSHIP',
   'POVERTY MITIGATION',
   'CONFERENCES',
   'YOUTH EMPLOYMENT'],
  ['YOUTH',
   'ENTREPRENEURSHIP',
   'YOUTH EMPLOYMENT',
   'EUROPE',
   'NORTH AMERICA',
   'ECONOMIES IN TRANSITION',
   'YOUTH POLICY',
   'AIDS PREVENTION',
   'POVERTY MITIGATION',
   'INFORMATION TECHNOLOGY'],
  ['MATERNAL WELFARE',
   'CHILD WELFARE',
   'UKRAINE',
   'SOCIALLY DISADVANTAGED CHILDREN'],
  ['YOU

In [4]:
print(len(outcry['jobno']))

159054


In [6]:
indices = [i for i, x in enumerate(outcry['keywords']) if x == []]

In [7]:
print(len(indices))

36894


In [8]:
import pandas as pd
outta = pd.DataFrame.from_dict(outcry,orient='index')
outta = outta.transpose()
outta.drop(outta.index[indices],inplace=True,)
outta = outta.reset_index(drop=True)
s = pd.Series(range(len(outta)))
outta.set_index(s,inplace=True,append = True)
outta


,,symbol,jobno,dat,language,keywords,paths,text
0,0,TRANS/WP.29/1999/14/ADD.1,G9920252,19900202,English,"[MOTOR VEHICLE EQUIPMENT, ANTI-THEFT DEVICES, ...",Downloads/ENGLISH/UNv1.0-TEI/en/1990\trans\wp_...,UNITED. NATIONS. E. Economic and Social. Counc...
1,1,A/AC.96/SR.472,G9202845,19921116,English,"[ELECTION OF OFFICERS, REFUGEE ASSISTANCE, REF...",Downloads/ENGLISH/UNv1.0-TEI/en/1992\a\ac_96\s...,Distr. GENERAL. A/AC.96/SR.472. 16 November 19...
2,2,A/CN.4/452,N9327323,19920510,English,"[INTERNATIONAL CRIMES, INTERNATIONAL CRIMINAL ...",Downloads/ENGLISH/UNv1.0-TEI/en/1992\a\cn_4\45...,Distr. GENERAL. A/CN.4/452. 10 May 1993. ENGLI...
3,3,A/CONF.157/LACRM/14,G9312404,19920225,English,"[HUMAN RIGHTS, WOMEN'S RIGHTS, PROSTITUTION, H...",Downloads/ENGLISH/UNv1.0-TEI/en/1992\a\conf_15...,Distr. GENERAL. A/CONF.157/LACRM/14. 25 Februa...
4,4,A/CONF.157/PC/56/ADD.1,G9214513,19921116,English,"[HUMAN RIGHTS, CONFERENCES, WORK ORGANIZATION]",Downloads/ENGLISH/UNv1.0-TEI/en/1992\a\conf_15...,Distr. GENERAL. A/CONF.157/PC/56/Add.1. 16 Nov...
5,5,A/CONF.172/7/ADD.2,G9401627,19920422,English,"[NATURAL DISASTERS, POVERTY, POPULATION GROWTH...",Downloads/ENGLISH/UNv1.0-TEI/en/1992\a\conf_17...,Distr. GENERAL. A/CONF.172/7/Add.2. 22 April 1...
6,6,CAT/C/SR.128,G9214490,19921118,English,"[PERIODIC REPORTS, PRISONER TREATMENT, DETAINE...",Downloads/ENGLISH/UNv1.0-TEI/en/1992\cat\c\sr_...,Distr. GENERAL. CAT/C/SR.128. 18 November 1992...
7,7,CAT/C/SR.130,G9214534,19921123,English,"[PRISONER TREATMENT, PERIODIC REPORTS, LIBYA, ...",Downloads/ENGLISH/UNv1.0-TEI/en/1992\cat\c\sr_...,Distr. GENERAL. CAT/C/SR.130. 23 November 1992...
8,8,CAT/C/SR.136,G9214642,19921126,English,"[WORK ORGANIZATION, CONFERENCES, HUMAN RIGHTS,...",Downloads/ENGLISH/UNv1.0-TEI/en/1992\cat\c\sr_...,Distr. GENERAL. CAT/C/SR.136. 26 November 1992...
9,9,CAT/C/SR.124/ADD.1,G9214435,19921118,English,"[DISAPPEARANCE OF PERSONS, PERIODIC REPORTS, P...",Downloads/ENGLISH/UNv1.0-TEI/en/1992\cat\c\sr_...,Distr. GENERAL. CAT/C/SR.124/Add.1. 18 Novembe...


In [9]:
outta['language'].fillna("English",inplace=True)

In [10]:
outta.insert(5, "Category",str, True) 

In [11]:
outta.columns

Index(['symbol', 'jobno', 'dat', 'language', 'keywords', 'Category', 'paths',
       'text'],
      dtype='object')

In [14]:
outta.to_csv("english2.csv",index=False)

In [96]:
outta['Category'] = "Others"
outta["Category"][outta['keywords'].str.contains(('PEACEKEEPING OPERATIONS') , na=False,regex=False)] = "Peace"
outta["Category"][outta['keywords'].str.contains(('TROOP WITHDRAWAL') , na=False,regex=False)] = "Peace"
outta["Category"][outta['keywords'].str.contains(('NUCLEAR DISARMAMENT, ') , na=False,regex=False)] = "Nuclear"
outta["Category"][outta['keywords'].str.contains(('NUCLEAR NON-PROLIFERATION') , na=False,regex=False)] = "Nuclear"
outta["Category"][outta['keywords'].str.contains(('NUCLEAR WEAPON TESTS') , na=False,regex=False)] = "Nuclear"
outta["Category"][outta['keywords'].str.contains(('NUCLEAR WEAPONS') , na=False,regex=False)] = "Nuclear"
outta["Category"][outta['keywords'].str.contains(('DISARMAMENT NEGOTIATIONS') , na=False,regex=False)] = "Disarment"
outta["Category"][outta['keywords'].str.contains(('DISARMAMENT') , na=False,regex=False)] = "Disarment"
outta["Category"][outta['keywords'].str.contains(('DISARMAMENT AGREEMENTS') , na=False,regex=False)] = "Disarment"
outta["Category"][outta['keywords'].str.contains(('CEASEFIRES') , na=False,regex=False)] = "Disarment"
outta["Category"][outta['keywords'].str.contains(('ARMED INCIDENTS') , na=False,regex=False)] = "War"
outta["Category"][outta['keywords'].str.contains(('WEAPONS OF MASS DESTRUCTION') , na=False,regex=False)] = "War"
outta["Category"][outta['keywords'].str.contains(('LANDMINES') , na=False,regex=False)] = "War"
outta["Category"][outta['keywords'].str.contains(('CONSULTATIONS') , na=False,regex=False)] = "Consultations"
outta["Category"][outta['keywords'].str.contains(('NEGOTIATION') , na=False,regex=False)] = "Negotiation"
outta["Category"][outta['keywords'].str.contains(('SANCTIONS') , na=False,regex=False)] = "Sanctions"
outta["Category"][outta['keywords'].str.contains(('HUMANITARIAN ASSISTANCE') , na=False,regex=False)] = "Assistance"
outta["Category"][outta['keywords'].str.contains(('FISSIONABLE MATERIALS') , na=False,regex=False)] = "Materials"
outta["Category"][outta['keywords'].str.contains(('OUTER SPACE') , na=False,regex=False)] = "Space"
#outta.info()

C:\Users\AlexVaqueroRamirez\Anaconda3\lib\site-packages\pandas\core\generic.py:8682: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  self._update_inplace(new_data)
C:\Users\AlexVaqueroRamirez\Anaconda3\lib\site-packages\IPython\core\interactiveshell.py:3296: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  exec(code_obj, self.user_global_ns, self.user_ns)


In [97]:
outta

,,symbol,jobno,dat,language,keywords,Category,paths,text
29,29,E/CN.4/SUB.2/1992/SR.35,G9213878,19920902,English,"[SANCTIONS, CIVILIAN PERSONS, HUMANITARIAN ASS...",Assistance,Downloads/ENGLISH/UNv1.0-TEI/en/1992\e\cn_4\su...,Distr. GENERAL. E/CN.4/Sub.2/1992/SR.35. 2 Sep...
37,37,A/47/1000,N9344925,19930813,English,"[NEGOTIATION, POLITICAL CONDITIONS, HAITI, HAI...",Negotiation,Downloads/ENGLISH/UNv1.0-TEI/en/1993\a\47\1000...,UNITED. NATIONS. Distr. GENERAL. A/47/1000. S/...
38,38,A/47/1001,N9345598,19930820,English,"[MAPS, PEACEKEEPING OPERATIONS, CYPRUS, FINANC...",Peace,Downloads/ENGLISH/UNv1.0-TEI/en/1993\a\47\1001...,UNITED. NATIONS. Distr. GENERAL. A/47/1001. 20...
41,41,A/47/1004,N9346991,19930826,English,"[PEACEKEEPING OPERATIONS, CYPRUS, FINANCING]",Peace,Downloads/ENGLISH/UNv1.0-TEI/en/1993\a\47\1004...,UNITED. NATIONS. Distr. GENERAL. A/47/1004. 26...
45,45,A/47/1008,N9348297,19930902,English,"[CEASEFIRES, ISRAEL, PALESTINE QUESTION, MIDDL...",Disarment,Downloads/ENGLISH/UNv1.0-TEI/en/1993\a\47\1008...,UNITED. NATIONS. Distr. GENERAL. A/47/1008. S/...
47,47,A/47/1010,N9348537,19930903,English,"[SANCTIONS, POLITICAL CONDITIONS, HAITI, HAITI...",Sanctions,Downloads/ENGLISH/UNv1.0-TEI/en/1993\a\47\1010...,UNITED. NATIONS. Distr. GENERAL. A/47/1010. S/...
52,52,A/47/1015,N9349297,19930910,English,"[PEACEKEEPING OPERATIONS, CYPRUS, BUDGET CONTR...",Peace,Downloads/ENGLISH/UNv1.0-TEI/en/1993\a\47\1015...,UNITED. NATIONS. Distr. GENERAL. A/47/1015. 10...
56,56,A/47/1019,N9350235,19930915,English,"[CEASEFIRES, ISRAEL, TERRITORIES OCCUPIED BY I...",Negotiation,Downloads/ENGLISH/UNv1.0-TEI/en/1993\a\47\1019...,UNITED. NATIONS. Distr. GENERAL. A/47/1019. S/...
57,57,A/47/1020,N9350448,19930916,English,"[CEASEFIRES, ISRAEL, TERRITORIES OCCUPIED BY I...",Disarment,Downloads/ENGLISH/UNv1.0-TEI/en/1993\a\47\1020...,UNITED. NATIONS. Distr. GENERAL. A/47/1020. S/...
58,58,A/47/1021,N9350871,19930917,English,"[CEASEFIRES, ISRAEL, PALESTINE QUESTION, TERRI...",Disarment,Downloads/ENGLISH/UNv1.0-TEI/en/1993\a\47\1021...,UNITED. NATIONS. Distr. GENERAL. A/47/1021. 17...


In [98]:
outta.groupby('Category')[['jobno']].count()

,jobno
Category,
Assistance,4568
Consultations,1841
Disarment,2484
Materials,266
Negotiation,1937
Nuclear,1276
Peace,8880
Sanctions,3690
Space,714


In [93]:
outta = outta[outta.Category != 'Others']

In [94]:

outta.to_csv("english.csv",index=False)

In [95]:
print(len(outta))

30223


In [ ]:
outta